# init data

In [2]:
import pandas as pd
import numpy as np
from typing import List, Tuple, Dict, Any

seed = 18022004
np.random.seed(seed)

In [1]:
data_prefix: str = 'data'
repo_prefix: str = f'{data_prefix}/repos'

# data_name_template: str = '500_{}_sample_dataset.parquet'
# data_types = ['log', 'test', 'others']

# sampled_data_name: Dict[str, str] = {data_type: data_name_template.format(data_type) for data_type in data_types}

# log_df: pd.DataFrame = pd.read_parquet(f'{data_prefix}/{sampled_data_name['log']}', engine = 'pyarrow')
# test_df: pd.DataFrame = pd.read_parquet(f'{data_prefix}/{sampled_data_name['test']}', engine = 'pyarrow')
# others_df: pd.DataFrame = pd.read_parquet(f'{data_prefix}/{sampled_data_name['others']}', engine = 'pyarrow')

# data_name = 'migration_others_method_no_code.parquet'
data_name = 'migration_others_dataset_cutoff_test_512_output.parquet'

# load data

In [3]:
data_df: pd.DataFrame = pd.read_parquet(f'{data_prefix}/{data_name}', engine = 'pyarrow')

# init parser

In [5]:
from tree_sitter import Language, Parser
import tree_sitter_java as tsjava
import difflib

from tqdm import tqdm

# Initialize the parser and set the Java language
JAVA_LANGUAGE = Language(tsjava.language())
parser = Parser(JAVA_LANGUAGE)

# preview

In [4]:
data_df.columns

Index(['id', 'fromLib', 'toLib', 'repoName', 'repoOwner', 'repoSplitName',
       'prevCommit', 'startCommit', 'endCommit', 'fileName', 'startCode',
       'endCode', 'diff', 'startCommitChanges', 'endCommitChanges',
       'total_added', 'total_removed', 'total_position', 'detailed_changes',
       'startCode_cleaned', 'endCode_cleaned', 'diff_cleaned',
       'lib_percentage'],
      dtype='object')

In [4]:
data_df

,id,fromLib,toLib,repoName,prevCommit,startCommit,endCommit,fileName,startCommitChanges,endCommitChanges,...,method_before_parameters,method_after_parameters,method_before_signature,method_after_signature,method_before_signature_no_mod,method_after_signature_no_mod,added,removed,predicted,prompt
0,48781,org.springframework.security.oauth:spring-secu...,io.jsonwebtoken:jjwt,gravitee-io_graviteeio-access-management,3b353da5a24425e4e40d325433c00f49bbf87ab9,0b18ada3207991cdfd70a551c56af125d2a0efe1,0b18ada3207991cdfd70a551c56af125d2a0efe1,gravitee-am-management-api/gravitee-am-managem...,+io.jsonwebtoken:jjwt\n+org.springframework.se...,+io.jsonwebtoken:jjwt\n+org.springframework.se...,...,,,@Override public boolean upgrade(),@Override public boolean upgrade(),boolean upgrade(),boolean upgrade(),domainService.findById(ADMIN_DOMAIN)\n.switchI...,try {\n\nDomain adminDomain = domainService.fi...,```java\nimport io.jsonwebtoken.lang.Assert;\n...,<｜begin▁of▁sentence｜>### Instruction:\nyou're ...
1,35650,net.sf.trove4j:trove4j,com.carrotsearch:hppc,owlcs_owlapi,cbf63f7f9e3de211999c97955a6bed9add474736,7d603c302dccccfa80d8572a2825142431908eb0,8bac5ab5d2ce4d7660c40ef5d337cfb4a8713064,parsers/src/main/java/org/semanticweb/owlapi/k...,+com.carrotsearch:hppc\n-net.sf.trove4j:trove4j,+com.github.vsonnier:hppcrt\n-net.sf.trove4j:t...,...,"char[] cbuf, int off, int len","char[] cbuf, int off, int len","@Override public int read(char[] cbuf, int off...","@Override public int read(char[] cbuf, int off...","int read(char[] cbuf, int off, int len)","int read(char[] cbuf, int off, int len)",int numBytesToRead = numBytesInBuffer - off;,int numBytesToRead = numBytesInBuffer -off;,```java\n@Override\npublic int read(char[] cbu...,<｜begin▁of▁sentence｜>### Instruction:\nyou're ...
2,35706,net.sf.trove4j:trove4j,com.carrotsearch:hppc,owlcs_owlapi,cbf63f7f9e3de211999c97955a6bed9add474736,7d603c302dccccfa80d8572a2825142431908eb0,8bac5ab5d2ce4d7660c40ef5d337cfb4a8713064,parsers/src/main/java/org/semanticweb/owlapi/k...,+com.carrotsearch:hppc\n-net.sf.trove4j:trove4j,+com.github.vsonnier:hppcrt\n-net.sf.trove4j:t...,...,OWLDataMinCardinality ce,OWLDataMinCardinality ce,@Override public void visit(OWLDataMinCardinal...,@Override public void visit(OWLDataMinCardinal...,void visit(OWLDataMinCardinality ce),void visit(OWLDataMinCardinality ce),writeOpenBracket();\nwrite(AT_LEAST);\nwrite(c...,"writeCardinality(AT_LEAST, ce);",```java\npublic void visit(OWLDataMinCardinali...,<｜begin▁of▁sentence｜>### Instruction:\nyou're ...
3,9871,com.alibaba:fastjson,com.fasterxml.jackson.core:jackson-core,lodsve_lodsve-framework,2a5a86372e7bd2271026c48cc64aa000b504926b,ba9b070e2c50de9916e4e7e282a7cb7dc4e64970,ba9b070e2c50de9916e4e7e282a7cb7dc4e64970,message-mvc/src/main/java/message/mvc/commons/...,+com.fasterxml.jackson.core:jackson-core\n-cos...,+com.fasterxml.jackson.core:jackson-core\n-cos...,...,HttpServletResponse response,HttpServletResponse response,public void setResponse(HttpServletResponse re...,public void setResponse(HttpServletResponse re...,void setResponse(HttpServletResponse response),void setResponse(HttpServletResponse response),,,```java\npublic void setResponse(HttpServletRe...,<｜begin▁of▁sentence｜>### Instruction:\nyou're ...
4,24376,com.sun.jersey.contribs:jersey-multipart,org.glassfish.jersey.media:jersey-media-multipart,ngageoint_mrgeo,a42baf6a6e9e1d52beb89bf2e842c9e428c4dd6c,b658dcdc752a47bddb92d36e6c64facbd6e08fd2,b658dcdc752a47bddb92d36e6c64facbd6e08fd2,mrgeo-services/mrgeo-services-wms/src/test/jav...,+org.glassfish.jersey.media:jersey-media-json-...,+org.glassfish.jersey.media:jersey-media-json-...,...,String paramName,String paramName,private void testGetMapMissingParam(String par...,private void testGetMapMissingParam(String par...,void testGetMapMissingParam(String paramName),void testGetMapMissingParam(String paramName),"WebTarget webResource = target().path(""/wms"")\...","WebResource webResource = resource().path(""/wm...",```java\nprivate vo

In [108]:
def print_tree(node, indent = 0):
    """
    Recursively prints the tree structure of a tree-sitter node.

    :param node: The current tree-sitter node.
    :param source_code: The source code being parsed, as bytes.
    :param indent: The current indentation level for pretty-printing.
    """
    # Extract node information
    node_type = node.type
    node_text = node.text.decode('utf-8')

    node_text = [line for line in node_text.split('\n') if line.strip()]

    # Print the node details with indentation
    print(f"{'  ' * indent}{node_type}: '{node_text}'")

    # Recursively print child nodes
    for child in node.children:
        print_tree(child, indent + 1)

def extract_method_details(node, class_name, source_code):
    method_details = {
        'class_name': class_name,
        'body': '',
        'modifiers': '',
        'return_type': '',
        'name': '',
        'parameters': '',
        'signature': '',
    }

    start_byte, end_byte = node.start_byte, node.end_byte
    method_details['body'] = source_code[start_byte:end_byte].decode('utf-8', errors = 'replace')

    for child in node.children:
        if (child.type == 'modifiers'):
            method_details['modifiers'] = ' '.join([modifier.text.decode('utf-8') for modifier in child.children])
        elif ('type' in child.type):  # Return type
            method_details['return_type'] = child.text.decode('utf-8')
        elif (child.type == 'identifier'):  # Method name
            method_details['name'] = child.text.decode('utf-8')
        elif (child.type.endswith('parameters')):  # Parameter list
            param_string = ', '.join([param.text.decode('utf-8') for param in child.children if param.type.endswith('parameter')])
            method_details['parameters'] = param_string

    method_details['signature'] = f'{method_details["modifiers"]} {method_details["return_type"]} {method_details["name"]}({method_details["parameters"]})'
    method_details['signature_no_mod'] = f'{method_details["return_type"]} {method_details["name"]}({method_details["parameters"]})'

    return method_details

def extract_methods_with_body(java_code):
    # print('java_code :',java_code)

    def has_errors(node):
        if node.type == 'ERROR':
            print(node.text.decode('utf-8'))
            return True
        return any(has_errors(child) for child in node.children)

    def dfs_find_methods(node: Any, encoded_code, class_context: List[str] = None, ) -> List[Dict[str, Any]]:
        '''Perform DFS to find all method_declaration nodes and their enclosing class hierarchy.'''
        if class_context is None:
            class_context = []

        method_details = []

        # If the node is a class, update the class context
        if node.type == 'class_declaration':
            class_name = None
            for child in node.children:
                if child.type == 'identifier':  # Class name
                    class_name = child.text.decode('utf-8')
                    break
            if class_name:
                class_context.append(class_name)

        # If the node is a method, extract its details with the full class hierarchy
        if node.type == 'method_declaration':
            full_class_name = '.'.join(class_context)  # Concatenate class names to show the hierarchy
            method_details.append(extract_method_details(node = node, class_name = full_class_name, source_code = encoded_code,))

        # Recursively process all children
        for child in node.children:
            method_details.extend(dfs_find_methods(child, encoded_code, class_context[:]))  # Pass a copy of class context

        # If the node is a class, pop the class name after processing its children
        if node.type == 'class_declaration' and class_context:
            class_context.pop()

        return method_details

    try:
        encoded_code = java_code.encode('utf-8')
        tree = parser.parse(encoded_code)
        root_node = tree.root_node

        if (has_errors(root_node)):
            raise Exception('Parsing errors found in the code')

        return dfs_find_methods(root_node, encoded_code)
    except Exception as e:
        print('Loi :' ,e)
        return None

# Function to remove comments
def remove_comments(java_code: str) -> str:
    # Parse the code
    tree = parser.parse(java_code.encode('utf-8'))
    root_node = tree.root_node

    # Gather ranges of comment nodes
    comment_ranges = []
    def visit_node(node):
        if node.type in {'line_comment', 'block_comment'}:
            comment_ranges.append((node.start_byte, node.end_byte, node.type))
        for child in node.children:
            visit_node(child)

    visit_node(root_node)

    # Remove comments by excluding their byte ranges
    result_code = bytearray(java_code, 'utf-8')
    for start, end, comment_type in reversed(comment_ranges):  # Reverse to avoid shifting indices
        if comment_type == 'block_comment':
            # Replace block comment with spaces
            result_code[start:end] = b' ' * (end - start + 1)
            # del result_code[start:end]
        else:
            # Remove line comments entirely
            result_code[start:end] = b' ' * (end - start + 1)
        # del result_code[start:end]

    return result_code.decode('utf-8').strip()

def diff_methods(methods_start, methods_end):
    '''
    Compare methods based on their full dictionaries (e.g., name, signature, body).
    '''
    # Normalize methods for comparison
    def normalize_methods(methods):
        res = []
        for method in methods:
            sub_method = {}

            sub_method['class_name'] = method['class_name'].strip()
            sub_method['name'] = method['name'].strip()
            sub_method['body'] = method['body'].strip()
            sub_method['modifiers'] = method['modifiers'].strip()
            sub_method['return_type'] = method['return_type'].strip()
            sub_method['parameters'] = method['parameters'].strip()
            sub_method['signature'] = method['signature'].strip()
            sub_method['signature_no_mod'] = method['signature_no_mod'].strip()

            res.append(sub_method)

        return res

    normalized_start = normalize_methods(methods_start)
    normalized_end = normalize_methods(methods_end)
    # normalized_start = [{key: method[key].strip() if isinstance(method[key], str) else method[key] for key in method} for method in methods_start]
    # normalized_end = [{key: method[key].strip() if isinstance(method[key], str) else method[key] for key in method} for method in methods_end]

    # Convert lists of methods to sets of frozensets for comparison
    set_start = set(frozenset(item.items()) for item in normalized_start)
    set_end = set(frozenset(item.items()) for item in normalized_end)

    # Determine differences
    removed_methods = [dict(items) for items in (set_start - set_end)]  # Methods in start but not in end
    added_methods = [dict(items) for items in (set_end - set_start)]    # Methods in end but not in start
    unchanged_methods = [dict(items) for items in (set_start & set_end)]  # Methods in both

    return {
        'removed': removed_methods,
        'added': added_methods,
        'unchanged': unchanged_methods,
    }

def check_same_methods(method1, method2):
    '''
    Check if two methods are the same based on their full dictionaries.
    '''
    return method1['signature_no_mod'] == method2['signature_no_mod']

In [117]:
for id in range(len(data_df)):
    # if (id != 8219):
    #     continue

    id = 8219
    sample = data_df.iloc[id]

    start_code = sample['startCode']
    end_code = sample['endCode']

    with open('before.txt', 'w') as f:
        f.write(start_code)
    with open('after.txt', 'w') as f:
        f.write(end_code)

    start_code_cleaned = remove_comments(start_code)
    end_code_cleaned = remove_comments(end_code)

    with open('before_no_comment.txt', 'w') as f:
        f.write(start_code_cleaned)
    with open('after_no_comment.txt', 'w') as f:
        f.write(end_code_cleaned)

    # with open('before.txt', 'w') as f:
    #     f.write(start_code)

    # print(start_code)

    method_start = extract_methods_with_body(start_code)
    method_end = extract_methods_with_body(end_code)

    # print(method_start)
    # print(method_end)

    # print(len(method_start))
    # print(len(method_end))

    break

# start_code

_(Class<? extends SubView> cls) {
Loi : Parsing errors found in the code
_(Class<? extends SubView> cls) {
Loi : Parsing errors found in the code


In [7]:
for id in range(len(data_df)):
    sample = data_df.iloc[1]

    print(sample['prompt'])
    print('-' * 50)
    print(sample['predicted'])

    break

<｜begin▁of▁sentence｜>### Instruction:
you're a software engineer working on a project. ONLY RESPOND WITH CODE, NO COMMENTS, IMPORTS, TEXT, NO EXPLAIN.
rewrite below method from library "net.sf.trove4j:trove4j" to "com.carrotsearch:hppc".
```
@Override
		public int read(char[] cbuf, int off, int len) throws IOException {
			int numCharsOutstandingInString = _size - _position;
			
			if (numCharsOutstandingInString == 0) {
				return -1;
			}
			
			int numBytesInBuffer = cbuf.length;
			int numBytesToRead = numBytesInBuffer -off;
			numBytesToRead = numBytesToRead > len ? len : numBytesToRead;
			
			if (numBytesToRead > numCharsOutstandingInString) {
				numBytesToRead = numCharsOutstandingInString;
			}
			
			_string.getChars(_position, _position + numBytesToRead, cbuf, off);
			
			_position += numBytesToRead;
			
			return numBytesToRead;
		}
```

### Response:

--------------------------------------------------
```java
@Override
public int read(char[] cbuf, int off, int len) throw